# CREA LA TUA AI DA ZERO
## Costruiamo una rete neurale per distinguere gatti e pesci

---

## 1. INTRODUZIONE AL PROBLEMA: COME INSEGNARE AD UN COMPUTER A DISTINGUERE UN PESCE DA UN GATTO?

Immagina di mostrare a un bambino tante foto di gatti e tante foto di pesci. Dopo un po', il bambino sarà in grado di riconoscere le differenze: i gatti hanno baffi, orecchie a punta e sono pelosi, mentre i pesci hanno squame, pinne e vivono nell'acqua.

Il computer, però, non "vede" come noi. Per lui un'immagine non è altro che una griglia di numeri (i pixel). Ogni pixel ha un valore che rappresenta il colore.

Come facciamo a insegnare a un computer a riconoscere pattern complessi come "baffi" o "pinne" partendo solo da numeri?

La risposta è: attraverso l'esperienza! Mostriamo al computer migliaia di esempi di gatti e pesci, e lui impara a trovare da solo le caratteristiche distintive. Questo processo si chiama **machine learning**.

In questo notebook costruiremo passo passo un programma che impara a distinguere gatti da pesci.

---

## 2. LE RETI NEURALI: IMPARARE A PREVEDERE I RISULTATI BASANDOSI SU CALCOLI MATEMATICI

Una rete neurale è come un circuito di "neuroni" artificiali collegati tra loro. Ogni neurone riceve input, esegue un semplice calcolo matematico e produce un output.

La rete impara regolando la forza (chiamata **peso**) delle connessioni tra i neuroni. Se la rete sbaglia a riconoscere un'immagine, aggiusta leggermente questi pesi per fare meglio la prossima volta.

Matematicamente, una rete neurale è una funzione complessa che trasforma i pixel di input in una probabilità di output: "questo è un gatto" o "questo è un pesce".

---

## 3. LE RETI NEURALI IN PYTHON CON PYTORCH

Per costruire reti neurali in Python, useremo **PyTorch**, una libreria potente e flessibile.

PyTorch ci permette di:
- Eseguire calcoli veloci sulla GPU
- Calcolare automaticamente le derivate (fondamentale per l'apprendimento)
- Costruire reti neurali con poche righe di codice

### 3.1 INTRODUZIONE A PYTORCH: TENSORI, RETI NEURALI

I **tensori** sono l'equivalente PyTorch degli array NumPy, ma con superpoteri: possono essere spostati sulla GPU e tengono traccia delle operazioni per calcolare le derivate.

Una rete neurale in PyTorch si definisce creando una classe che eredita da `torch.nn.Module`.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

In [ ]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
simplenet = SimpleNet()

---

## 4. CREAZIONE DEL DATASET: CLASSI DATALOADER E DATASET CON TORCHVISION

Per addestrare la nostra rete, abbiamo bisogno di dati. Useremo **torchvision**, un pacchetto di PyTorch con dataset e trasformazioni predefinite.

I concetti chiave:
- **Dataset**: contiene i dati (immagini e etichette)
- **DataLoader**: carica i dati in batch durante l'addestramento
- **Transform**: applica trasformazioni alle immagini (ridimensionamento, normalizzazione, ecc.)

Per questo corso, creeremo un dataset personalizzato di gatti e pesci, contenente circa 1000 immagini totali.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
    

img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image)

batch_size=64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

---

## 5. CREARE IL MODELLO: OTTIMIZZATORE, FUNZIONE DI ERRORE, TRAINING LOOP

Ora costruiamo il nostro classificatore gatto/pesce. Useremo:

- **Funzione di errore (Loss function)**: misura quanto il modello sbaglia. Per classificazione useremo `CrossEntropyLoss`.
- **Ottimizzatore (Optimizer)**: aggiorna i pesi del modello per ridurre l'errore. Useremo `Adam` o `SGD`.
- **Training loop**: il ciclo dove:
  1. Prendiamo un batch di immagini
  2. Le passiamo al modello (forward pass)
  3. Calcoliamo l'errore
  4. Calcoliamo i gradienti (backward pass)
  5. Aggiorniamo i pesi

In [ ]:
# funzione di train

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

---

## 6. SCARICARE IL MODELLO E RIUTILIZZARLO PER L'INFERENZA

Una volta addestrato, il modello ha imparato a distinguere gatti da pesci. Possiamo salvarlo su disco e ricaricarlo quando serve, senza doverlo riaddestrare ogni volta.

Questo è utile per:
- Distribuire il modello ad altri
- Usare il modello in produzione
- Continuare l'addestramento in un secondo momento

In [ ]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")    
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)   

---

## CONCLUSIONE

Congratulazioni! Hai costruito la tua prima AI in grado di distinguere gatti da pesci.

Abbiamo visto:
- Come i computer vedono le immagini (come numeri)
- Le basi delle reti neurali
- Come usare PyTorch per costruire e addestrare un modello
- Come caricare dati, addestrare e fare inferenza

Questo è solo l'inizio. Puoi applicare queste stesse tecniche a problemi molto più complessi!